# Modules and session

In [49]:
import pyspark
from pyspark.sql import SparkSession
from pyspark.sql.functions import col,isnan, when, count
from textblob import TextBlob

In [2]:
spark=SparkSession.builder.appName('Translate').getOrCreate()

22/09/22 19:10:08 WARN Utils: Your hostname, LAPTOP-7NHMLFJ5 resolves to a loopback address: 127.0.1.1; using 172.18.14.112 instead (on interface eth0)
22/09/22 19:10:08 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


22/09/22 19:10:11 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


# Data

In [3]:
data=spark.read.option('header', 'true').csv('files.csv',inferSchema= True)

In [4]:
data.show(5)

+--------------------+------------+--------------------+
|        wav_filename|wav_filesize|          transcript|
+--------------------+------------+--------------------+
|audios/4da6b70e-0...|      219064|y aquí en dos pal...|
|audios/8c2ab30b-0...|      271910|cuando los consej...|
|audios/ca73c951-c...|       64520|su mujer con la c...|
|audios/067c4606-7...|       84222|y otros que se po...|
|audios/49a08f90-3...|       77316|al oír mis pasos ...|
+--------------------+------------+--------------------+
only showing top 5 rows



# Prepossessing

## Missings

In [5]:
data.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in data.columns]).show()

+------------+------------+----------+
|wav_filename|wav_filesize|transcript|
+------------+------------+----------+
|           0|           0|         0|
+------------+------------+----------+



In [69]:
data.select([count(when(col(c).contains('None') | \
                            col(c).contains('NULL') | \
                            (col(c) == '' ) | \
                            col(c).isNull() | \
                            isnan(c), c 
                           )).alias(c)
                    for c in data.columns]).show()

+------------+------------+----------+
|wav_filename|wav_filesize|transcript|
+------------+------------+----------+
|           0|           0|         0|
+------------+------------+----------+



## Traducción

In [7]:
rdd= data.rdd

In [8]:
rdd.take(2)

[Row(wav_filename='audios/4da6b70e-0108-4f75-80ae-3d71f1dd2c2b.wav', wav_filesize=219064, transcript='y aquí en dos palotadas hemos encontrado robustas columnas donde apoyar la grandiosa fábrica de su alcurnia'),
 Row(wav_filename='audios/8c2ab30b-0fd4-41c3-9724-3b15f2ee2c27.wav', wav_filesize=271910, transcript='cuando los consejeros escucharon aquello quedaron estremecidos y se dijeron dios ha prohibido que padres se casen con sus hijas')]

In [10]:
rdd.map(lambda x: (x,1)).take(2)

[(Row(wav_filename='audios/4da6b70e-0108-4f75-80ae-3d71f1dd2c2b.wav', wav_filesize=219064, transcript='y aquí en dos palotadas hemos encontrado robustas columnas donde apoyar la grandiosa fábrica de su alcurnia'),
  1),
 (Row(wav_filename='audios/8c2ab30b-0fd4-41c3-9724-3b15f2ee2c27.wav', wav_filesize=271910, transcript='cuando los consejeros escucharon aquello quedaron estremecidos y se dijeron dios ha prohibido que padres se casen con sus hijas'),
  1)]

In [13]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize

nltk.download('stopwords')
from nltk.corpus import stopwords

[nltk_data] Downloading package punkt to /home/bluterplay/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bluterplay/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [14]:
stopword_list  = stopwords.words('spanish')

In [16]:
rdd_word_list= rdd.map(lambda x : (x[0],x[1],x[2].split(" ")))

In [87]:
rdd_stopwords= rdd_word_list.map(lambda x: (x[0],x[1],[i for i in x[2] if i not in stopword_list])) \
                            .map(lambda x:(x[0],x[1], " ".join(x[2]))).filter(lambda x:x[2]!='')

In [88]:
rdd_stopwords.take(2)

[('audios/4da6b70e-0108-4f75-80ae-3d71f1dd2c2b.wav',
  219064,
  'aquí dos palotadas encontrado robustas columnas apoyar grandiosa fábrica alcurnia'),
 ('audios/8c2ab30b-0fd4-41c3-9724-3b15f2ee2c27.wav',
  271910,
  'consejeros escucharon aquello quedaron estremecidos dijeron dios prohibido padres casen hijas')]

In [136]:
def trans(x):  
    print(x)
    blob= TextBlob(x)
    return str(blob.translate(from_lang='es', to= 'en' ))

In [ ]:
test= rdd_stopwords.take()

In [124]:
test=rdd_stopwords.filter(lambda x: x[1]<25000)

In [95]:
for i in ["Hola", "como","estas"]:
    print(translate(i))

Hello
What
these


In [129]:
spark

In [135]:
test.map(lambda x: TextBlob(x[2]).translate(from_lang='es', to= 'en' )).take(2)

22/09/22 20:56:46 ERROR Executor: Exception in task 0.0 in stage 80.0 (TID 181)
org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 668, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 85, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 173, in _read_with_length
    return self.loads(obj)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 471, in loads
    return cloudpickle.loads(obj, encoding=encoding)
ModuleNotFoundError: No module named 'textblob'

	at org.apache.spark.api.python.BasePythonRunner$ReaderIterator.handlePythonException(PythonRunner.scala:559)
	at org.apache.spark.api.python.PythonRunner$$anon$3.read(PythonRunner.scala:765)
	at org.apache.spark.api.python.PythonRu

Py4JJavaError: An error occurred while calling z:org.apache.spark.api.python.PythonRDD.runJob.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 80.0 failed 1 times, most recent failure: Lost task 0.0 in stage 80.0 (TID 181) (172.18.14.112 executor driver): org.apache.spark.api.python.PythonException: Traceback (most recent call last):
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 668, in main
    func, profiler, deserializer, serializer = read_command(pickleSer, infile)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/worker.py", line 85, in read_command
    command = serializer._read_with_length(file)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 173, in _read_with_length
    return self.loads(obj)
  File "/opt/spark/python/lib/pyspark.zip/pyspark/serializers.py", line 471, in loads
    return cloudpickle.loads(obj, encoding=encoding)
ModuleNotFoundError: No module named 'textblob'

	at org.apache.spar